## Objective : Analyze the dataset to calculate financial metrics for each account, rank them, and provide a top 20 list.

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

     

df = pd.read_csv(r'C:\DATASCIENCE\RonyProject\Primetrade.ai\TRADES_CopyTr_90D_ROI.csv')

In [64]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


## Handle Missing Values

In [66]:
data_clean = df.dropna(subset=['Trade_History'])

In [67]:
data_clean.shape

(149, 2)

In [68]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       149 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.5+ KB


## Parse the JSON-like Trade_History Column

### The Trade_History column contains JSON-like objects. You need to convert these into a structured format for further analysis.

In [69]:
data_clean['Trade_History'] = data_clean['Trade_History'].apply(ast.literal_eval)

# Check the first few parsed entries
data_clean['Trade_History'].head()

C:\Users\Rony\AppData\Local\Temp\ipykernel_19428\2913990028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Trade_History'] = data_clean['Trade_History'].apply(ast.literal_eval)


0    [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1    [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2    [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3    [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4    [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
Name: Trade_History, dtype: object

## Extract Trade Details

### Now, extract the trade details (e.g., symbol, price, quantity, etc.) from Trade_History for each Port_ID.

In [70]:
trade_data_list = []

# Loop through each row and extract trade details
for idx, row in data_clean.iterrows():
    port_id = row['Port_IDs']
    trade_history = row['Trade_History']

    for trade in trade_history:
        trade_data_list.append({
            'Port_ID': port_id,
            'time': trade.get('time'),
            'symbol': trade.get('symbol'),
            'side': trade.get('side'),
            'price': trade.get('price'),
            'quantity': trade.get('quantity'),
            'realizedProfit': trade.get('realizedProfit', 0),
            'positionSide': trade.get('positionSide')
        })

# Convert the list to a DataFrame
trade_data_df = pd.DataFrame(trade_data_list)

# Convert 'time' to datetime format
trade_data_df['time'] = pd.to_datetime(trade_data_df['time'], unit='ms')

# Check the extracted data
trade_data_df.head()

,Port_ID,time,symbol,side,price,quantity,realizedProfit,positionSide
0,3925368433214965504,2024-06-20 16:07:36,SOLUSDT,BUY,132.53700,1988.05500,0.0,LONG
1,3925368433214965504,2024-06-20 16:06:58,DOGEUSDT,BUY,0.12182,1398.98088,0.0,LONG
2,3925368433214965504,2024-06-20 16:06:58,DOGEUSDT,BUY,0.12182,197.47022,0.0,LONG
3,3925368433214965504,2024-06-20 16:06:56,DOGEUSDT,BUY,0.12182,16.56752,0.0,LONG
4,3925368433214965504,2024-06-20 16:06:56,DOGEUSDT,BUY,0.12182,92.21774,0.0,LONG


## Calculate Metrics

### 1. Return on Investment (ROI)

In [78]:
def calculate_roi(x):
    investment = (x['price'] * x['quantity']).sum()
    profit = x['realizedProfit'].sum()
    # Prevent division by zero
    return 0 if investment == 0 else (profit / investment) * 100

### 2. Profit and Loss (PnL)

In [72]:
pnl_df = trade_data_df.groupby('Port_ID')['realizedProfit'].sum().reset_index(name='PnL')
     

### 3. Win Rate

In [73]:
trade_data_df['win'] = trade_data_df['realizedProfit'] > 0

# Calculate win rate for each Port_ID
win_rate_df = trade_data_df.groupby('Port_ID')['win'].mean().reset_index(name='Win_Rate')

# Calculate total win positions
win_positions_df = trade_data_df.groupby('Port_ID')['win'].sum().reset_index(name='Win_Positions')

# Calculate total positions
total_positions_df = trade_data_df.groupby('Port_ID')['win'].count().reset_index(name='Total_Positions')


### 4. Sharpe Ratio

In [79]:
def calculate_sharpe(x, risk_free_rate=0.01):
    returns = x['realizedProfit']
    if len(returns) < 2:  # Need at least 2 points
        return 0
    std = returns.std()
    if std == 0:  # Prevent division by zero
        return 0
    return (returns.mean() - risk_free_rate) / std

### 5. Maximum Drawdown (MDD)

In [83]:
def calculate_mdd(returns):
    if len(returns) < 2:
        return 0
    cumulative_returns = (1 + returns).cumprod()
    running_max = cumulative_returns.expanding().max()
    drawdowns = (cumulative_returns - running_max) / running_max
    return abs(drawdowns.min()) if not drawdowns.empty else 0



## Ranking Accounts

### Combine all the metrics into one DataFrame and rank the accounts:

In [84]:
# Main metrics calculation function
def calculate_account_metrics(trade_data_df):
    metrics = []

    for port_id in trade_data_df['Port_ID'].unique():
        port_trades = trade_data_df[trade_data_df['Port_ID'] == port_id]

        # Calculating basic metrics
        roi = calculate_roi(port_trades)
        pnl = port_trades['realizedProfit'].sum()
        win_rate = (port_trades['realizedProfit'] > 0).mean() * 100
        total_trades = len(port_trades)
        sharpe = calculate_sharpe(port_trades)
        mdd = calculate_mdd(port_trades['realizedProfit'])
        # Storing the metrics
        metrics.append({
            'Port_ID': port_id,
            'ROI': roi,
            'PnL': pnl,
            'Win_Rate': win_rate,
            'Total_Trades': total_trades,
            'Sharpe_Ratio': sharpe,
            'MDD': mdd
})

 # Converting to DataFrame
    metrics_df = pd.DataFrame(metrics)

    # Normalize metrics between 0 and 1
    cols_to_normalize = ['ROI', 'PnL', 'Win_Rate', 'Sharpe_Ratio']
    for col in cols_to_normalize:
        min_val = metrics_df[col].min()
        max_val = metrics_df[col].max()
        if max_val != min_val:
            metrics_df[f'{col}_norm'] = (metrics_df[col] - min_val) / (max_val - min_val)
        else:
            metrics_df[f'{col}_norm'] = metrics_df[col] / max_val if max_val != 0 else 0

    # Calculate composite score (with equal weights)
    metrics_df['Score'] = (
        metrics_df['ROI_norm'] * 0.25 +
        metrics_df['PnL_norm'] * 0.25 +
        metrics_df['Win_Rate_norm'] * 0.25 +
        metrics_df['Sharpe_Ratio_norm'] * 0.25
    )

    # Calculate rank (1 is best)
    metrics_df['Rank'] = metrics_df['Score'].rank(ascending=False, method='min')

    return metrics_df.sort_values('Rank')

metrics_df = calculate_account_metrics(trade_data_df)

# Sort by rank and select top 20
top_20_accounts = metrics_df.sort_values('Rank').head(20)

# Preview top 20 accounts
top_20_accounts[['Port_ID', 'Score', 'Rank']]

# Displaying results with rounded numbers for better readability
display_columns = ['Port_ID', 'ROI', 'PnL', 'Win_Rate', 'Total_Trades', 'Score', 'Rank']
print(top_20_accounts[display_columns].round(2))


c:\Users\Rony\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)


                 Port_ID    ROI       PnL  Win_Rate  Total_Trades  Score  Rank
46   3977234346014419201  36.53   2427.29     39.76            83   0.54   1.0
130  3826087012661391104   0.00    532.66     58.33           108   0.47   2.0
35   3986814617275053313   0.09  16337.46     78.22          3554   0.46   3.0
128  3956048468100538880   0.00   1373.56     71.43            28   0.46   4.0
21   3891020560590657281   6.80   2856.30     64.76           437   0.46   5.0
126  3999240873283311617   0.10  42574.47     52.32          4522   0.43   6.0
41   3983074113875692800  18.79   2424.83     32.56            43   0.42   7.0
109  4039279455324236544   1.84   1038.81     55.35           327   0.41   8.0
82   4020204877254599680   0.00  71998.86     32.94          6050   0.40   9.0
119  3886752488982104320   0.00   7195.18     74.54          1249   0.39  10.0
85   4022641794255717633   0.35   2176.83     43.52          6052   0.39  11.0
117  3943533600390906881   0.00   2361.42     54.80 

## Saving Results

In [85]:
metrics_df.to_csv(r'metrics.csv', index=False)

# Save top 20 accounts to CSV
top_20_accounts.to_csv(r'top_20_accounts.csv', index=False)

     